In [ ]:
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from torch import nn, optim
from sklearn.metrics import root_mean_squared_error
import wandb
import plotly.express as px

In [ ]:
data= px.data.tips()

data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
#Encode Categorical columns
cat_cols= list(data.select_dtypes(include= 'object').columns)

for col in cat_cols:
    encoder= LabelEncoder()
    data[col]= encoder.fit_transform(data[col])

In [ ]:
#split the dataset into train set and test set
X=data.drop(columns=['tip'])
y=data['tip']

#do train test split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=11)


In [ ]:
#Scale the data
scaler=StandardScaler() #init the scaler

scaler.fit(X_train)     #scale the train set
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)


In [ ]:
# convert the data to tensor
X_train_tensor = torch.tensor(data= X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(data= X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(data= y_train.values, dtype=torch.float32).unsqueeze(dim=1)
y_test_tensor = torch.tensor(data= y_test.values, dtype=torch.float32).unsqueeze(dim=1)

print(f'shape of X: {X_train.shape}')
print(f'shape of X: {y_train.shape}')

shape of X: (183, 6)
shape of X: (183,)


In [ ]:
# setup the model architecture

class Tip_Model(nn.Module):

  # constructor
  def __init__(self, number_of_columns: int):
    super().__init__()
    self.fc1 = nn.Linear(in_features=number_of_columns, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=50)
    self.fc3 = nn.Linear(in_features=50, out_features=10)
    self.out = nn.Linear(in_features=10, out_features=1)
    self.relu = nn.ReLU()

  def forward(self, X: torch.Tensor) -> torch.Tensor:
    result = self.relu(self.fc1(X))
    result = self.relu(self.fc2(result))
    result = self.relu(self.fc3(result))
    result = self.out(result)

    return result

In [ ]:
# init the model and model hyper parameters
model = Tip_Model(number_of_columns=X_train.shape[1])
LEARNING_RATE = 1e-4
optimizer = optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()
EPOCHS = 10000

In [ ]:
#Set up experiment tracking
wandb.init(
    project= "Tip Model",
    name= "Regression MLP for Tip Model",
    config= {
        "network type": "MLP with 3 hidden layers",
        "criterion": "MSELoss",
        "optimizer": "Adam",
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCHS,
        "batch_size": 32
        }
)

In [ ]:
# setup the training looop
loss_list = []
test_loss_list= []
model.train()
for epoch in range(EPOCHS):
  # forward pass
  train_preds = model(X_train_tensor)
  loss = criterion(train_preds, y_train_tensor)

  # back prop
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  #Validation loop
  with torch.no_grad():
    test_preds = model(X_test_tensor)
    test_loss = criterion(test_preds, y_test_tensor)
    test_loss_list.append(test_loss.item())


  if epoch % 1000 == 0:
    print(f"complete epoch: {(epoch//1000) + 1}/{EPOCHS/1000}...loss : {loss.item()}, Test loss: {test_loss.item()}")
    loss_list.append(loss.item())


    #Log the experiment
    wandb.log(
        data= {
            'epoch': epoch+1,
            "train_loss": loss.item(),
            "test_loss": test_loss.item()
        }
    )



wandb.finish()


complete epoch: 1/10.0...loss : 10.255498886108398, Test loss: 9.76400375366211
complete epoch: 2/10.0...loss : 0.9907993674278259, Test loss: 1.0099629163742065
complete epoch: 3/10.0...loss : 0.7638685703277588, Test loss: 0.9345665574073792
complete epoch: 4/10.0...loss : 0.5276282429695129, Test loss: 1.0946186780929565
complete epoch: 5/10.0...loss : 0.3432976305484772, Test loss: 1.611227035522461
complete epoch: 6/10.0...loss : 0.2556007206439972, Test loss: 2.158714532852173
complete epoch: 7/10.0...loss : 0.21198025345802307, Test loss: 2.384223699569702
complete epoch: 8/10.0...loss : 0.19063244760036469, Test loss: 2.479063034057617
complete epoch: 9/10.0...loss : 0.16795629262924194, Test loss: 2.7210898399353027
complete epoch: 10/10.0...loss : 0.14835834503173828, Test loss: 3.1105754375457764


epoch,▁▂▃▃▄▅▆▆▇█
test_loss,█▁▁▁▂▂▂▂▂▃
train_loss,█▂▁▁▁▁▁▁▁▁
epoch,9001
test_loss,3.11058
train_loss,0.14836


In [ ]:
train_preds = train_preds.detach().numpy()
train_preds = train_preds.squeeze()
print(f'Train Rmse: {root_mean_squared_error(y_train, train_preds)}')

Train Rmse: 0.3618332166208955


In [ ]:
#Model Evaluation
model.eval()

with torch.no_grad():
  test_preds = model(X_test_tensor)
  test_preds = test_preds.squeeze()
  test_preds = test_preds.detach().numpy()

In [ ]:
print(f'Test Rmse: {root_mean_squared_error(y_test, test_preds)}')

Test Rmse: 1.9187736414146561
